# SWAT+

This notebook is an example of BOA running a real environmental model, SWAT+. 

In the `swat` folder you will find a SAASBO configuration file that pulls the SWAT+ possible parameters from the SWAT+ `cal_parms.cal` file to create a high dimensional optimization problem to run on a SLURM cluster as well as a highly parallelized SLURM configuration file. Both configurations are too computationally expensive to run here as an example, but the `swat` folder contains the necessary files to run the model on your own machine or cluster. Both configurations are shown below.

Here we will run a simpler example, under the `swat/config_local_test.yaml` configuration file. This configuration file is a simplified version of the `swat` configuration file, with 8 parameters, 2 constraints, and 10 trials.

In [1]:
# This notebook uses hidden cells to import and display things so
# that it can be ran regullary to make sure the documentation
# is up to date and not broken (as opposed to a markdown file
# that would have an exmaple written once and get out of date
# as the code base changes)

# These hidden cells are only responsible for rerunning
# the documentation to ensure it is correct.
# the actual relavent part of the documentation
# are the non hidden parts

In [2]:
import pathlib
import boa

from IPython.display import Code

boa_paper_dir = pathlib.Path().resolve().parent
r_dir = boa_paper_dir / "swat"
config_path = r_dir / "config_local_test.yaml"
run_model_script = r_dir / "run_model.R"

config_saasbo_path = r_dir / "config_auto_saasbo.yaml"
config_parr_path = r_dir / "config_parr.yaml"

## Configuration File

### SAASBO

config_auto_saasbo.yaml

In [3]:
Code(config_saasbo_path, language="yaml")

{% set pathlib = load_py("pathlib") %}
{% set builtins = load_py("builtins") %}
{% set cal_params_f = pathlib.Path("./TxtInOut/cal_parms.cal") %}
objective:
    metrics:
        -   name: flo_out_NSE
            minimize: False

parameters:
    start_date:
        type: fixed
        value: "2008-01-01"
    end_date:
        type: fixed
        value: "2018-01-01"
    years_skip:
        type: fixed
        value: 5
    {% set f = builtins.open(cal_params_f) -%}
    {% for line in f -%}
    {% if loop.index0 >= 3 -%}
    {% set name1, name2, abs_min, abs_max, unit = line.split() %}
    {{ name1 }}.{{ name2 }}:
        type: range
        bounds: [{{ abs_min }}, {{ abs_max }}]
        value_type: float
    {%- endif %}
    {%- endfor %}
    {% do f.close() %}

scheduler:
    n_trials: 500
    max_pending_trials: 25
    run_trials_in_batches: true
generation_strategy:
    num_initialization_trials: 100
    max_parallelism_override: 25
    use_saasbo: true
script_options:
    run_model: sbatch --time=3:00:00 slurm_launch_run_model.sh
    exp_name: swat_plus_slurm_auto_saasbo

The SAASBO configuration file tells BOA to dynamically grab the parameters from the SWAT+ `cal_parms.cal` file and create a high dimensional optimization problem to run on a SLURM cluster.

When BOA gets the parameters from the `cal_parms.cal` file, it will create a parameter for each line in the file. The `cal_parms.cal` file is a text file that contains the SWAT+ parameters that can be optimized as well as their bounds they can be. Each line in the file is a parameter that can be optimized. Optimizing every parameter blindly with their maximum bounds is obviously not the most sound setup, but it demonstrates the flexibility of BOA to handle high dimensional optimization problems.

### Parallel

config_parr.yaml

In [4]:
Code(config_parr_path, language="yaml")

{% set max_parr = 25 %}

objective:
    metrics:
        -   name: flo_out_NSE
            minimize: False

parameters:
    start_date: "2013-01-01"
    end_date: "2018-01-01"
    years_skip: 0
    cn3_swf.hru:
        type: range
        bounds: [0, 1]
        value_type: float
    esco.hru:
        type: range
        bounds: [0, 1]
        value_type: float
    epco.hru:
        type: range
        bounds: [0, 1]
        value_type: float
    perco.hru:
        type: range
        bounds: [0, 1]
        value_type: float
    snofall_tmp.hru:
        type: range
        bounds: [-5, 5]
        value_type: float
    snomelt_tmp.hru:
        type: range
        bounds: [-5, 5]
        value_type: float
    snomelt_max.hru:
        type: range
        bounds: [0, 10]
        value_type: float
    snomelt_min.hru:
        type: range
        bounds: [0, 10]
        value_type: float

parameter_constraints:
    - snomelt_min.hru <= snomelt_max.hru
    - snofall_tmp.hru <= snomelt_tmp.hru

scheduler:
    total_trials: 200
    max_pending_trials: {{ max_parr }}
    run_trials_in_batches: true
generation_strategy:
    steps:
        -   model: SOBOL
            num_trials: 50
            max_parallelism: {{ max_parr }}
        -   model: GPEI
            num_trials: -1
            max_parallelism: {{ max_parr }}
script_options:
    run_model: sbatch --time=3:00:00 --output={{ trial_dir }}/%j.log slurm_launch_run_model.sh
    exp_name: swat_plus_{{ max_parr }}_parr_200_trials

## Run Model Wrapper Script

run_model.R

In [5]:
Code(run_model_script)

# (linux tested, does not run on Mac, SWATrunR says it runs with Windows, so probably Windows is ok)

# load in any libraries and modules we need
library(SWATrunR)
library(jsonlite)
library(dplyr)
library(here)
library(readr)
library(hydroGOF)

project_path <- here("swat", "TxtInOut")
trial_dir <- commandArgs(trailingOnly=TRUE)[1]
data <- read_json(path=file.path(trial_dir, "parameters.json"))

start_date <- data$start_date
end_date <- data$end_date
years_skip <- data$years_skip
data <- within(data, rm(start_date, end_date, years_skip))

par_comb = c()
for (name in names(data)) {
    par_comb[paste(name, " | change = absval")] = data[[name]]
}

sim <- run_swatplus(
    project_path=project_path,
    start_date = start_date,
    end_date = end_date,
    years_skip = years_skip,
    run_path=trial_dir,
    parameter = par_comb,
    output = define_output(
            file = 'channel_sd_day',
            variable = 'flo_out',
            unit = 1)
        )

obs <- read_csv(here("swat","obs_data.csv"))
obs <- obs %>% filter(between(date, min(sim$simulation$flo_out$date), as.Date(end_date)))

flow_obs <- obs %>% filter(variable == "flow_cms") %>% within(rm(variable))
flow_sim <- sim$simulation$flo_out %>% filter(date %in% flow_obs$date)

# NSE (Nash–Sutcliffe model efficiency coefficient)
nse <- NSE(sim=flow_sim$run_1, obs=flow_obs$value)

jsn <- toJSON(list(
    flo_out_NSE=nse
), pretty = TRUE)
write(jsn, file.path(trial_dir, "output.json"))

## Running our script

To run our script we just need to pass the config file to BOA's CLI

```python
python -m boa --config-file path/to/config.yaml
```

or

```python
python -m boa -c path/to/config.yaml
```

In [7]:
output = !python -m boa -c {config_path}  # we capture ipython terminal output to python variable
o = "\n".join(ln for ln in output)  # it comes in as a list, we convert to string
o = o.replace(str(r_dir),  "[/path/to/your/dir/]")  # replace the actual dir with a stand in for privacy reasons
Code(o)

[INFO 07-13 16:55:05] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[FixedParameter(name='start_date', parameter_type=STRING, value='2017-01-01'), FixedParameter(name='end_date', parameter_type=STRING, value='2018-01-01'), FixedParameter(name='years_skip', parameter_type=INT, value=0), RangeParameter(name='cn3_swf.hru', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='esco.hru', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='epco.hru', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='perco.hru', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='snofall_tmp.hru', parameter_type=FLOAT, range=[-5.0, 5.0]), RangeParameter(name='snomelt_tmp.hru', parameter_type=FLOAT, range=[-5.0, 5.0]), RangeParameter(name='snomelt_max.hru', parameter_type=FLOAT, range=[0.0, 10.0]), RangeParameter(name='snomelt_min.hru', parameter_type=FLOAT, range=[0.0, 10.0])], parameter_constraints=[OrderConstraint(snomelt_min.hru <= snomelt_max.hru), OrderConstraint(snofall_tmp.hru <= snomelt_tmp.hru)]).
[INFO 07-13 16:55:05] ax.modelbridge.dispatch_utils: Using Models.GPEI since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 07-13 16:55:05] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=8 num_trials=None use_batch_trials=False
[INFO 07-13 16:55:05] ax.modelbridge.dispatch_utils: calculated num_initialization_trials=16
[INFO 07-13 16:55:05] ax.modelbridge.dispatch_utils: num_completed_initialization_trials=0 num_remaining_initialization_trials=16
[INFO 07-13 16:55:05] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 16 trials, GPEI for subsequent trials]). Iterations after 16 will take longer to generate due to model-fitting.
[INFO 07-13 16:55:05] Scheduler: `Scheduler` requires experiment to have immutable search space and optimization config. Setting property immutable_search_space_and_opt_config to `True` on experiment.
[INFO 2024-07-13 16:55:05,504 MainProcess MainThread] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: [/path/to/your/dir/]/swat_plus_local_20240713T165505
Scheduler File Path: [/path/to/your/dir/]/swat_plus_local_20240713T165505/scheduler.json
Optimization CSV File Path: [/path/to/your/dir/]/swat_plus_local_20240713T165505/optimization.csv
Start Time: 20240713T165505
Version: 0.10.2

##############################################

[INFO 07-13 16:55:05] Scheduler: Running trials [0]...
[INFO 2024-07-13 16:55:05,554 MainProcess ThreadPoolExecutor-0_0] boa: Rscript run_model.R
[INFO 2024-07-13 16:55:05,559 MainProcess ThreadPoolExecutor-0_0] boa: {'config_path': PosixPath('[/path/to/your/dir/]/config_local_test.yaml'), 'config_dir': PosixPath('[/path/to/your/dir/]'), 'config_dir_name': 'swat', 'config_file_name': 'config_local_test.yaml', 'cn3_swf.hru': 0.8839095700532198, 'esco.hru': 0.6621499573811889, 'epco.hru': 0.9161975039169192, 'perco.hru': 0.3118354044854641, 'snofall_tmp.hru': -0.6253641005605459, 'snomelt_tmp.hru': 4.472801284864545, 'snomelt_max.hru': 9.241793220862746, 'snomelt_min.hru': 2.187449336051941, 'start_date': '2017-01-01', 'end_date': '2018-01-01', 'years_skip': 0, 'trial_dir': PosixPath('[/path/to/your/dir/]/swat_plus_local_20240713T165505/000000')}
[INFO 2024-07-13 16:55:05,560 MainProcess ThreadPoolExecutor-0_0] boa: Rscript run_model.R
[INFO 07-13 16:55:06] Scheduler: Running trials [1]...
[INFO 2024-07-13 16:55:06,645 MainProcess ThreadPoolExecutor-1_0] boa: Rscript run_model.R
[INFO 2024-07-13 16:55:06,648 MainProcess ThreadPoolExecutor-1_0] boa: {'config_path': PosixPath('[/path/to/your/dir/]/config_local_test.yaml'), 'config_dir': PosixPath('[/path/to/your/dir/]'), 'config_dir_name': 'swat', 'config_file